In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

<h3> Establecemos el número de ciudades </h3>

In [3]:
# Tamaño de ciudades
num_ciudades=1000
# Solución aleatoria
solucion = np.random.permutation(num_ciudades)
#print(solucion)

<h3> Cargamos el archivo con las ciudades </h3>

In [4]:
from StringIO import StringIO
df = pd.read_table('ar9152.tsp', engine='python',sep=' ', skiprows=7, skipfooter=1, names=['x','y'])

<h3> Eliminamos el último elemento </h3>

In [5]:
print df.tail()
# Borro el último
df = df[:-1]
#print df.tail()

               x           y
9148  34411.1111  58798.3333
9149  34813.3333  58475.2778
9150  34878.0556  58135.8333
9151  34917.2222  58386.9444
9152  34976.9444  58686.9444


In [6]:
posiciones=[(float(x),float(y)) for x, y in zip(df['x'],df['y'])]

<h3> Creamos la matriz posiciones y seleccionamos las primeras 1000 ciudades </h3>

In [7]:
pos = np.asarray(posiciones)
max_pos = pos.max(0)
min_pos = pos.min(0)
ratio_pos = max_pos - min_pos
pos = (pos-min_pos)/ratio_pos
posiciones = map(tuple, pos)
posiciones= map(tuple,pos[:1000])
#print(posiciones)

<h3> Calculamos la distancia euclidea </h3>

In [8]:
from scipy.spatial.distance import euclidean as distance

def get_matrix_costes(posiciones):
    num = len(posiciones)
    costes = np.array([distance(posiciones[c1],posiciones[c2]) for c1 in range(num) for c2 in range(num)])
    return costes.reshape((num, num))

In [9]:
costes = get_matrix_costes(posiciones)

<h3>Función fitness, que calcula la distancia empleada por 'solución' dado 'costes'</h3>

In [10]:
def fitness(solucion, costes):
    sum = 0
    
    for i in range(num_ciudades-1):
        c1 = solucion[i]
        c2 = solucion[i+1]
        sum += costes[c1,c2]
    # Volver a la ciudad inicial
    ciudad_final = solucion[num_ciudades-1]
    ciudad_origen = solucion[0]
    sum += costes[ciudad_final, ciudad_origen]
    return sum

In [11]:
# Evaluamos la distancía del recorrido obtenido con la función aleatoria
#solucion, fitness(solucion, costes)
fitness(solucion, costes)

295.01014324305379

<h3>Algoritmo de búsqueda aleatoria </h3>

In [12]:
def random_sol(num):
    return np.random.permutation(num)

In [13]:
from sys import maxint

def random_search(costes, evaluations):
    num = costes.shape[0]
    best = maxint
    best_sol = []
    
    for r in range(evaluations):
        solucion = random_sol(num)
        fitness_sol = fitness(solucion, costes)
        if (fitness_sol < best):
            best = fitness_sol
            best_sol = solucion
            
    return best_sol

<h3> Realizamos el algoritmo greedy </h3>

In [16]:
def solucion_greedy(costes):
    
    # Creamos un vector que almacena los id de las ciudades por visitar
    pendientes= x= np.arange(1000)
    
    # Eliminamos la primera ciudad
    pendientes=np.delete(pendientes,0)
    
    solucion= []
    
    ciudadActual=0
    
    # Añadimos el primer elemento
    solucion.append(ciudadActual)
    
    # Mientras queden ciudades pendientes
    while  0 < len(pendientes):
    
        # Obtenemos todas las distancias a las ciudades desde la ciudad c1
        distancias=costes[ciudadActual]
        
        # Escogemos de esas distancias sólo las ciudades que queden por recorrer
        distancias_destino = distancias[pendientes]
        
        # Copiamos las distancias destino para poder obtener 3 veces la ciudad más cercana
        # sin modificar el original
        distancias_destino2 = distancias_destino[:]
        # Pendientes2 almacena el identificador de la ciudad a la que hace referencia distancias_destino2
        pendientes2 = pendientes[:]
        
        # Inicializamos el vector que contiene las 3 ciudades que pueden ser elegidas
        idCiudadesCercanas= []
        
        # id1 contiene el id del vector que contiene las ciudades pendientes
        id1= distancias_destino2.argmin()
        # ciudad1 contiene el id de la ciudad
        ciudad1=pendientes2[id1]
        # Eliminamos la ciudad de pendientes2
        pendientes2=np.delete(pendientes2,id1)
        # Eliminamos el indice ciudad1 del vector distancias_destino2
        distancias_destino2=np.delete(distancias_destino2,id1)
        idCiudadesCercanas.append(ciudad1)
        
        # Controlamos que queden ciudades
        if (len(distancias_destino2) > 0):
            # Cogemos el indice del menor valor del vector
            id2= distancias_destino2.argmin();
            # ciudad2 contiene el id de la ciudad
            ciudad2=pendientes2[id2]
            # Eliminamos la ciudad de pendientes2
            pendientes2=np.delete(pendientes2,id2)
            # Eliminamos el indice ciudad2 del vector distancias_destino2
            distancias_destino2=np.delete(distancias_destino2,id2)
            idCiudadesCercanas.append(ciudad2)
        
        # Controlamos que queden ciudades
        if (len(distancias_destino2) > 0):
            # Cogemos el indice del menor valor del vector
            id3= distancias_destino2.argmin();
            # ciudad2 contiene el id de la ciudad
            ciudad3=pendientes2[id3]
            # Eliminamos la ciudad de pendientes2
            pendientes2=np.delete(pendientes2,id3)
            # Eliminamos el indice ciudad2 del vector distancias_destino3
            distancias_destino2=np.delete(distancias_destino2,id3)
            idCiudadesCercanas.append(ciudad3)
        
        # Si solo queda una ciudad escogemos la primera
        if (len(idCiudadesCercanas) == 1):
            ciudadActual=idCiudadesCercanas[0]
        
        # Una vez que tenemos las distancias de las 3 ciudades escogemos una aleatoriamente
        if (len(idCiudadesCercanas) == 2):
            ciudadActual=idCiudadesCercanas[random.randint(0,1)]
        
        if (len(idCiudadesCercanas) == 3):
            ciudadActual=idCiudadesCercanas[random.randint(0,2)]
        
        # Añadimos la CiudadElegida a el camino seguido
        solucion.append(ciudadActual)
        
        # Busacamos el índice y eliminamos la ciudad
        pendientes=np.delete(pendientes,np.searchsorted(pendientes,ciudadActual, side="left"))
        
    # La solución está formada por el conjunto de vertices que son la solución
    return solucion

<h3> Tsp greedy ejecuta el algoritmo greedy con el criterio de parada especificado </h3>

In [17]:
from sys import maxint

def tsp_greedy(costes, evaluations):
    num = costes.shape[0]
    best = maxint
    best_sol = []
    
    for r in range(evaluations):
        solucion = solucion_greedy(costes)
        fitness_sol = fitness(solucion, costes)
        if (fitness_sol < best):
            best = fitness_sol
            best_sol = solucion
            
    return best_sol

<h3> Realizamos la comparación entre el algoritmo aleatorio y el greedy implementado  </h3>

In [18]:
distancia1=0
distancia2=0
    
for r in range(10):
    distancia1+= fitness(tsp_greedy(costes,5000),costes)
    distancia2+= fitness(random_search(costes,5000),costes)
    
distanciat1= distancia1/10
distanciat2= distancia2/10

print "La distancia del algoritmo devorador es: "+"%.2f" % distanciat1
print "La distancia del algoritmo aleatorio es: "+"%.2f" % distanciat2

La distancia del algoritmo devorador es: 19.96
La distancia del algoritmo aleatorio es: 287.88
